# Getting started

First, we enable the cluster to scale up. Note that if you run an auto-scaling cluster,
Google will suspend your nodes. Make sure to have the experiment prepared before running the commands.

The following is assumed ready:
* GKE/Kubernetes cluster (see also `terraform/terraform_notebook.ipynb`)
    * 2 nodes pools (default for system & dependencies, experiment pool)
* Docker image (including dataset, to speed-up starting experiments).
    * Within a bash shell
        * Make sure to have the `requirements-cpu.txt` installed (or `requirements-gpu.txt (in a virtual venv/conda environment). You can run `pip3 install -r requirements-cpu.txt`
    * First run the extractor (locally) `python3 -m fltk extractor configs/example_cloud_experiment.json`
        *  This downloads datasets to be included in the docker image.
    * Build the container `DOCKER_BUILDKIT=1 docker build --platform linux/amd64 . --tag gcr.io/$PROJECT_ID/fltk`
    * Push to your gcr.io repository `docker push gcr.io/$PROJECT_ID/fltk`


With that setup, first set some variables used throughout the experiment.


In [1]:
##################
### CHANGE ME! ###
##################
PROJECT_ID="eminent-parsec-362007"
CLUSTER_NAME="fltk-testbed-cluster"
DEFAULT_POOL="default-node-pool"
EXPERIMENT_POOL="medium-fltk-pool-1"
REGION="us-central1-c"
# alias gcloud=/home/yifan/.local/tools/google-cloud-sdk/bin/gcloud

# In case we do not yet have the credentials/kubeconfig
gcloud container clusters get-credentials $CLUSTER_NAME --region $REGION --project $PROJECT_ID

Fetching cluster endpoint and auth data.
kubeconfig entry generated for fltk-testbed-cluster.


Scale the default-node-pool up.

In [2]:
# These commands might take a while to complete.
gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
     --num-nodes 1 --region $REGION --quiet

Resizing fltk-testbed-cluster...done.                                          
Updated [https://container.googleapis.com/v1/projects/eminent-parsec-362007/zones/us-central1-c/clusters/fltk-testbed-cluster].


## Preparation
In case you have already tested something or ran another experiment, we have to remove the deployment of the Orchestrator. This will not delete any experiment data, as this persists on one of the ReadWriteMany PVCs.


Currently, the Orchestrator is deployed using a `Deployment` definition, a future version will replace this with a `Deployment` definition, to make this step unnecessary. For experiments this means the following:

1. A single deployment can exist at a single time in a single namespace. This includes 'completed' experiments.
2. For running batches of experiments, a BatchOrchestrator is provided.


ℹ️ This will not remove any data, but if your orchestrator is still/already running experiments, this will stop the deployment. Running training jobs will not be stopped, for this you can use `kubectl`. ConfigMaps created by the Orchestrator (to provide experiment configurations), will not be removed. See the commented code in the cell below.

In [3]:
# If you want to delete all pytorch trainjobs, uncomment the command below.
# kubectl delete pytorchjobs.kubeflow.org --all --namespace test

# If you want to delete all existing configuration map objects in a namespace, run teh command below
# kubectl delete configmaps --all --namespace test

helm uninstall -n test flearner

W1008 01:35:10.723238   31288 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Error: uninstall: Release not loaded: flearner: release: not found


: 1

## Install extractor

Deploy the TensorBoard service and persistent volumes, required for deployment of the orchestrator's chart.

In [4]:
helm upgrade --install -n test extractor ../charts/extractor -f ../charts/fltk-values.yaml \
    --set provider.projectName=$PROJECT_ID

W1008 01:37:13.185099   31914 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Release "extractor" has been upgraded. Happy Helming!
NAME: extractor
LAST DEPLOYED: Sat Oct  8 01:37:17 2022
NAMESPACE: test
STATUS: deployed
REVISION: 2
TEST SUITE: None
NOTES:
Get the FLTK extractors Tensorboard URL by running:

export POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")
echo http://localhost:6006/
kubectl -n test port-forward $POD_NAME 6006:6006


## Define experiment configuration files

Deployment of experiments is currently done through a Helm Deployment. A future release (™️) will rework this to a Job definition, as this allows to re-use the template more easily.


> The `EXPERIMENT_FILE` will contain the description of the experiments
> The `CLUSTER_CONFIG` will contain shared configurations for logging, Orchestrator configuration and replication information.

In [5]:
EXPERIMENT_FILE="../configs/distributed_tasks/example_arrival_config.json"
CLUSTER_CONFIG="../configs/example_cloud_experiment.json"

## Setup experiment variables
Next, we will deploy the experiments.


We provide a configuration file, `charts/fltk-values.yaml`, in here change the values under the `provider` block. Change `projectName` to your Google Cloud Project ID.

```yaml
provider:
    domain: gcr.io
    projectName: CHANGE_ME!
    imageName: fltk:latest
```

We use the `--set-file` flag for `helm`, as currently, Helm does not support using files outside of the chart root directory (in this case `charts/orchestrator`). Using `--set-file` we can dynamically provide these files. See also issue [here](https://github.com/helm/helm/issues/3276)


In [6]:
echo $EXPERIMENT_FILE
echo $CLUSTER_CONFIG
echo $PROJECT_ID

../configs/distributed_tasks/example_arrival_config.json
../configs/example_cloud_experiment.json
eminent-parsec-362007


In [7]:
helm uninstall -n test experiment-orchestrator
helm install -n test experiment-orchestrator ../charts/orchestrator -f ../charts/fltk-values.yaml \
    --set-file orchestrator.experiment=$EXPERIMENT_FILE,orchestrator.configuration=$CLUSTER_CONFIG \
    --set provider.projectName=$PROJECT_ID

W1008 01:39:06.908151   32453 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Error: uninstall: Release not loaded: experiment-orchestrator: release: not found
W1008 01:39:07.630393   32457 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
NAME: experiment-orchestrator
LAST DEPLOYED: Sat Oct  8 01:39:11 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
You successfully launched an experiment configuration on your cluster in test.

N.B. Make sure to collect all data after completing your experiment!
N.B. Re-installing the orchestrator WILL RESULT IN DELETION OF ALL TRAINJOBS and PODS!


In [17]:
kubectl get all -n test

W1008 01:46:47.410367   34508 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
NAME                                                         READY   STATUS    RESTARTS   AGE
pod/fl-extractor-7b79f57d98-tt8dz                            1/1     Running   0          14m
pod/fl-server                                                1/1     Running   0          7m34s
pod/nfs-server-nfs-server-provisioner-0                      1/1     Running   0          37m
pod/trainjob-33346b1e-e8b4-437f-a2fb-ee2042d4980f-master-0   1/1     Running   0          60s
pod/trainjob-33346b1e-e8b4-437f-a2fb-ee2042d4980f-worker-0   1/1     Running   0          60s
pod/trainjob-33784eca-66f8-4821-aa27-6c35a401e005-master-0   1/1     Running   0          4m
pod/trainjob-33784eca-66f8-4821-aa27-6c35a401e005-worker-0   1/1     Running   0          

In [8]:
helm ls -n=test

W1007 12:01:40.470686    8441 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
NAME                   	NAMESPACE	REVISION	UPDATED                                 	STATUS  	CHART                         	APP VERSION
experiment-orchestrator	test     	1       	2022-10-07 12:01:28.660963903 +0200 CEST	deployed	../docker-compose-gcloud-0.3.0	1.17.0     
extractor              	test     	5       	2022-10-07 12:01:16.723639247 +0200 CEST	deployed	fltk-extractor-0.1.0          	1.17.0     
nfs-server             	test     	1       	2022-10-05 16:05:30.825424619 +0200 CEST	deployed	nfs-server-provisioner-1.1.3  	2.3.0      


In [15]:
# To get logs from the orchestrator
kubectl logs -n test fl-server

W1008 01:45:11.093942   34089 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
10-07-2022 23:39:26 root         INFO     Loading file config/configuration.fltk.json
No argument path is provided.
10-07-2022 23:39:26 root         INFO     Starting in cluster mode.
10-07-2022 23:39:26 root         INFO     Starting with experiment replication: 0 with seed: 42
10-07-2022 23:39:26 root         INFO     Starting as Orchestrator
10-07-2022 23:39:26 root         INFO     Starting Orchestrator, initializing resources....
10-07-2022 23:39:26 root         INFO     Loading in cluster configuration file
10-07-2022 23:39:26 root         INFO     Pointing configuration to in cluster configuration.
10-07-2022 23:39:26 root         INFO     Starting cluster manager
10-07-2022 23:39:26 ClusterManager INFO     Spinning up cluster manag

10-07-2022 23:42:36 SimulatedArrivalGenerator INFO     Arrival train_job_0 arrives in 0 seconds
10-07-2022 23:42:37 Orchestrator INFO     Scheduling arrival of Arrival: 366bc88b-f62c-470b-9dd2-d7b272b1398e
10-07-2022 23:42:37 Orchestrator INFO     Deploying on cluster: 366bc88b-f62c-470b-9dd2-d7b272b1398e
10-07-2022 23:42:37 Orchestrator INFO     Still alive...
10-07-2022 23:42:42 Orchestrator INFO     Still alive...
10-07-2022 23:42:42 ResourceWatchDog INFO     Fetching pod information of cluster...
10-07-2022 23:42:46 SimulatedArrivalGenerator INFO     Creating task for train_job_0
10-07-2022 23:42:46 SimulatedArrivalGenerator INFO     Arrival train_job_0 arrives in 180 seconds
10-07-2022 23:42:47 Orchestrator INFO     Scheduling arrival of Arrival: 33784eca-66f8-4821-aa27-6c35a401e005
10-07-2022 23:42:47 Orchestrator INFO     Deploying on cluster: 33784eca-66f8-4821-aa27-6c35a401e005
10-07-2022 23:42:47 Orchestrator INFO     Still alive...
10-07-2022 23:42:52 Orchestrator INFO     S

In [18]:
# To get logs from learners (example)
kubectl logs -n test trainjob-9afc7e4d-e38a-4f8d-88f4-3d38727842f1-worker-0

# To get logs from learners (federated learning)
kubectl logs -n test trainjob-9afc7e4d-e38a-4f8d-88f4-3d38727842f1-master-0

W1008 01:47:35.389867   34721 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Defaulted container "pytorch" out of: pytorch, init-pytorch (init)
10-07-2022 23:44:32 root         INFO     Loading file config/configuration.fltk.json
10-07-2022 23:44:32 root         INFO     Starting in client mode
10-07-2022 23:44:32 root         INFO     Starting with host=trainjob-9afc7e4d-e38a-4f8d-88f4-3d38727842f1-master-0 and port=23456
10-07-2022 23:44:32 root         INFO     Initializing backend for training process: gloo
10-07-2022 23:44:32 torch.distributed.distributed_c10d INFO     Added key: store_based_barrier_key:1 to store for rank: 1
10-07-2022 23:44:32 torch.distributed.distributed_c10d INFO     Rank 1: Completed store-based barrier for key:store_based_barrier_key:1 with 2 nodes.
10-07-2022 23:44:32 root         INFO

## Copy experiment results from the extractor

Extractor holds the experiment results in the format that can be processedby TensorBoard.
In order to download it to the local machine, execute:

In [21]:
EXTRACTOR_POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")
kubectl cp -n test $EXTRACTOR_POD_NAME:logging ./logging

W1008 02:10:52.350628   40894 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
W1008 02:10:53.539496   40899 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke


In [24]:
ls ./logging

0


In [25]:
kubectl exec -ti -n test pod/$EXTRACTOR_POD_NAME -- curl localhost:8080

W1008 02:11:11.374570   40981 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
error: Internal error occurred: error executing command in container: failed to exec in container: failed to start exec "3509c48e84cb9243df6a62abcedcc27e9531f4aa50a1504963adb56d9271de2d": OCI runtime exec failed: exec failed: container_linux.go:380: starting container process caused: exec: "curl": executable file not found in $PATH: unknown


: 1

# Cleanup

## Removing orchestrator

In [26]:
helm uninstall -n test experiment-orchestrator

W1008 02:11:20.047576   41016 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "experiment-orchestrator" uninstalled


## Removing extractor

IMPORTANT: Removing extractor chart will result in deleting the already collected experiment results, stored in the NFS!

In [27]:
helm uninstall extractor -n test

W1008 02:11:26.921983   41059 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "extractor" uninstalled


## Wrapping up

To scale down the cluster nodepools, run the cell below. This will scale the node pools down and remove all the experiments deployed (on the cluster).

1. Experiments cannot be restarted.
2. Experiment logs will not persist deletion.


In [28]:
# This will remove all information and logs as well.
kubectl delete pytorchjobs.kubeflow.org --all-namespaces --all

gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
    --num-nodes 0 --region $REGION --quiet

gcloud container clusters resize $CLUSTER_NAME --node-pool $EXPERIMENT_POOL \
    --num-nodes 0 --region $REGION --quiet

W1008 02:11:37.373227   41102 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
pytorchjob.kubeflow.org "trainjob-023f24a1-ced9-441b-9dcc-ba79452a80ce" deleted
pytorchjob.kubeflow.org "trainjob-07a8af2d-d7b1-4d1a-a02e-e59e6e8ab4f2" deleted
pytorchjob.kubeflow.org "trainjob-29ea39a2-5748-4ede-8e19-3b326a05c6c1" deleted
pytorchjob.kubeflow.org "trainjob-33346b1e-e8b4-437f-a2fb-ee2042d4980f" deleted
pytorchjob.kubeflow.org "trainjob-33784eca-66f8-4821-aa27-6c35a401e005" deleted
pytorchjob.kubeflow.org "trainjob-366bc88b-f62c-470b-9dd2-d7b272b1398e" deleted
pytorchjob.kubeflow.org "trainjob-916ed6ec-1529-4396-b091-197133f02692" deleted
pytorchjob.kubeflow.org "trainjob-9afc7e4d-e38a-4f8d-88f4-3d38727842f1" deleted
pytorchjob.kubeflow.org "trainjob-c72b3b23-6865-4087-a2d4-9d8b9a06a8d7" deleted
Resizing fltk-testbed-cluster